# Retrieve World Bank Trade Data

Data comes from [https://wits.worldbank.org/witsapiintro.aspx]

Prereqs:
* Need to run "pip install pandaSDMX" to install parser to read SDMX data from a request

In [1]:
from pandasdmx import Request
import json
from pprint import pprint as pp

In [2]:
wits = Request('WBG_WITS')
flow_response = wits.dataflow()
flow_response.write().dataflow.head()

,name
dataflow,
DF_WITS_Tariff_TRAINS,WITS - UNCTAD TRAINS Tariff Data
DF_WITS_TradeStats_Development,WITS TradeStats Devlopment
DF_WITS_TradeStats_Tariff,WITS TradeStats Tariff
DF_WITS_TradeStats_Trade,WITS TradeStats Trade


In [3]:
trade_stats = wits.dataflow('DF_WITS_TradeStats_Trade')

In [4]:
trade_data_structure = trade_stats.dataflow.DF_WITS_TradeStats_Trade.structure()
trade_data_structure

DataStructureDefinition | TRADESTATS | Country Trade Statistics

In [5]:
trade_data_structure.dimensions.aslist()

[Dimension | FREQ,
 Dimension | REPORTER,
 Dimension | PARTNER,
 TimeDimension | TIME_PERIOD,
 Dimension | PRODUCTCODE,
 Dimension | INDICATOR]

Only interested in trade with the US

In [6]:
reporters = trade_stats.write().codelist.loc['REPORTER']
select_reporter = reporters.loc[['USA']]
select_reporter

,dim_or_attr,name
USA,D,United States


Looking for specific trade partners: Mexico, Canada, EU, China, Russia, World. We limit the definition of the EU to just the 11 countries that adopted the Euro as their currency in 1999.  I tried to use the partner code 'EUN' but there is no data in the with that partner code.

In [7]:
partners = trade_stats.write().codelist.loc['PARTNER']

#for index, row in partners.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

# Countries we are interested
partner_list = ['MEX','CAN','CHN','RUS']
          
# Countries which adopted the Euro as their currency in 1999
eu_list = ['AUT','BEL','FIN','FRA','DEU','IRL','ITA','LUX','NLD','PRT','ESP']
          
# Separate list for the lonely total partner representing the world 'WLD'
world_list = ['WLD']
          
select_partners = partners.loc[partner_list + eu_list + world_list]
select_partners

,dim_or_attr,name
MEX,D,Mexico
CAN,D,Canada
CHN,D,China
RUS,D,Russian Federation
AUT,D,Austria
BEL,D,Belgium
FIN,D,Finland
FRA,D,France
DEU,D,Germany
IRL,D,Ireland


Only product we are interested in is "all products" i.e. 'Total'

In [8]:
products = trade_stats.write().codelist.loc['PRODUCTCODE']

#for index, row in products.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

select_product = products.loc[['Total']]
select_product

,dim_or_attr,name
Total,D,All Products


Only looking for the import and export values. All values for these two indicators are in thousands of USD.

In [9]:
indicators = trade_stats.write().codelist.loc['INDICATOR']

#for index, row in indicators.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

select_indicators = indicators.loc[['MPRT-TRD-VL', 'XPRT-TRD-VL']]
select_indicators

,dim_or_attr,name
MPRT-TRD-VL,D,Import Trade Value (US$ Thousand)
XPRT-TRD-VL,D,Export Trade Value (US$ Thousand)


Only time frequency is 'annual'.

In [10]:
trade_stats.write().codelist.loc['FREQ'].head()

,dim_or_attr,name
FREQ,D,Freq
A,D,Annual


In [11]:
freq = 'A'
partners_list = select_partners.index.tolist()
reporter = select_reporter.index.tolist()[0]
product = select_product.index.tolist()[0]
indicator_list = select_indicators.index.tolist()

response = wits.data(resource_id = 'DF_WITS_TradeStats_Trade',
                 key={'FREQ': freq,
                      'REPORTER': reporter,
                      'PARTNER':  partners_list,
                      'PRODUCTCODE': product,
                      'INDICATOR': indicator_list},
                 params = {'startPeriod': '2000',
                           'endPeriod': '2017'},
                 dsd = trade_data_structure)

In [12]:
trade_df = response.write((s for s in response.data.series))
trade_df.rename(columns={'MPRT-TRD-VL': 'Imports', 'XPRT-TRD-VL': 'Exports'}, inplace=True)

import_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Imports') for cty in eu_list ]
import_column_eu = ('A', 'USA', 'EU', 'Total', 'Imports')
trade_df[import_column_eu] = trade_df[import_columns_to_sum].sum(axis=1)
trade_df.drop(import_columns_to_sum, axis=1, inplace=True)

export_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Exports') for cty in eu_list ]
export_column_eu = ('A', 'USA', 'EU', 'Total', 'Exports')
trade_df[export_column_eu] = trade_df[export_columns_to_sum].sum(axis=1)
trade_df.drop(export_columns_to_sum, axis=1, inplace=True)

known_cty_list = partner_list + [ 'EU' ]

known_import_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Imports') for cty in known_cty_list ]
import_column_other = ('A', 'USA', 'Other', 'Total', 'Imports')
import_column_world = ('A', 'USA', 'WLD', 'Total', 'Imports')
trade_df[import_column_other] = trade_df[import_column_world] - trade_df[known_import_columns_to_sum].sum(axis=1)

known_export_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Exports') for cty in known_cty_list ]
export_column_other = ('A', 'USA', 'Other', 'Total', 'Exports')
export_column_world = ('A', 'USA', 'WLD', 'Total', 'Exports')
trade_df[export_column_other] = trade_df[export_column_world] - trade_df[known_export_columns_to_sum].sum(axis=1)

trade_df.head()

FREQ                    A                                            \
REPORTER              USA                                             
PARTNER               CAN                         CHN                 
PRODUCTCODE         Total                       Total                 
INDICATOR         Imports       Exports       Imports       Exports   
TIME_PERIOD                                                           
2000         2.308161e+08  1.789200e+08  1.000129e+08  1.618468e+07   
2001         2.162341e+08  1.634211e+08  1.022673e+08  1.918194e+07   
2002         2.123984e+08  1.609182e+08  1.335104e+08  2.212751e+07   
2003         2.250353e+08  1.699202e+08  1.633139e+08  2.836772e+07   
2004         2.601613e+08  1.898788e+08  2.105001e+08  3.442769e+07   

FREQ                                                                \
REPORTER                                                             
PARTNER               MEX                         RUS                
PRODUCTCODE         Total                       Total                
INDICATOR         Imports       Exports       Imports      Exports   
TIME_PERIOD                                                          
2000         1.359231e+08  1.113386e+08  7.658130e+06  2092380.131   
2001         1.313347e+08  1.012951e+08  6.264073e+06  2716088.460   
2002         1.360251e+08  9.747019e+07  7.207030e+06  2396944.154   
2003         1.396867e+08  9.741174e+07  9.106465e+06  2447218.211   
2004         1.578819e+08  1.107312e+08  1.261872e+07  2960936.609   

FREQ                                                                 \
REPORTER                                                              
PARTNER               WLD                          EU                 
PRODUCTCODE         Total                       Total                 
INDICATOR         Imports       Exports       Imports       Exports   
TIME_PERIOD                                                           
2000         1.217933e+09  7.818307e+08  1.635439e+08  1.163625e+08   
2001         1.140900e+09  7.290804e+08  1.658866e+08  1.118365e+08   
2002         1.200096e+09  6.930683e+08  1.766811e+08  1.050110e+08   
2003         1.302834e+09  7.247366e+08  1.917040e+08  1.108980e+08   
2004         1.525304e+09  8.148444e+08  2.148414e+08  1.241318e+08   

FREQ                                     
REPORTER                                 
PARTNER             Other                
PRODUCTCODE         Total                
INDICATOR         Imports       Exports  
TIME_PERIOD                              
2000         5.799788e+08  3.569325e+08  
2001         5.189134e+08  3.306296e+08  
2002         5.342738e+08  3.051445e+08  
2003         5.739871e+08  3.156918e+08  
2004         6.693007e+08  3.527139e+08

In [13]:
# This line commented below should have worked but it doesn't.  You cannot use
# rename to rename columns with tuple names.  I had to use something a little
# more dangerous by explicitly set the columns attribute of the DataFrame

#new_column_names = { x: f"{x[2]}_{x[4]}" for x in list(trade_df) }
#trade_df.rename(columns=new_column_names, inplace=True)

trade_df.columns = [f"{x[2]}_{x[4]}" for x in trade_df.columns]
trade_df.head()

,CAN_Imports,CAN_Exports,CHN_Imports,CHN_Exports,MEX_Imports,MEX_Exports,RUS_Imports,RUS_Exports,WLD_Imports,WLD_Exports,EU_Imports,EU_Exports,Other_Imports,Other_Exports
TIME_PERIOD,,,,,,,,,,,,,,
2000,2.308161e+08,1.789200e+08,1.000129e+08,1.618468e+07,1.359231e+08,1.113386e+08,7.658130e+06,2092380.131,1.217933e+09,7.818307e+08,1.635439e+08,1.163625e+08,5.799788e+08,3.569325e+08
2001,2.162341e+08,1.634211e+08,1.022673e+08,1.918194e+07,1.313347e+08,1.012951e+08,6.264073e+06,2716088.460,1.140900e+09,7.290804e+08,1.658866e+08,1.118365e+08,5.189134e+08,3.306296e+08
2002,2.123984e+08,1.609182e+08,1.335104e+08,2.212751e+07,1.360251e+08,9.747019e+07,7.207030e+06,2396944.154,1.200096e+09,6.930683e+08,1.766811e+08,1.050110e+08,5.342738e+08,3.051445e+08
2003,2.250353e+08,1.699202e+08,1.633139e+08,2.836772e+07,1.396867e+08,9.741174e+07,9.106465e+06,2447218.211,1.302834e+09,7.247366e+08,1.917040e+08,1.108980e+08,5.739871e+08,3.156918e+08
2004,2.601613e+08,1.898788e+08,2.105001e+08,3.442769e+07,1.578819e+08,1.107312e+08,1.261872e+07,2960936.609,1.525304e+09,8.148444e+08,2.148414e+08,1.241318e+08,6.693007e+08,3.527139e+08


Now let's save the data as JSON so that we do not have to repeatedly make API calls to retrieve the data.

In [17]:
trade_csv = trade_df.to_csv()

# now write output to a file
with open('trade_data_file.csv', 'w') as outfile:
    outfile.write(trade_csv)